In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from zipfile import ZipFile 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
test_dir="C:/Users/ABHISHIKTA DHAR/Desktop/MISTU/dataset/test_set"
train_dir="C:/Users/ABHISHIKTA DHAR/Desktop/MISTU/dataset/training_set"

train_dir_cats = train_dir + '/cats'
train_dir_dogs = train_dir + '/dogs'
test_dir_cats = test_dir + '/cats'
test_dir_dogs = test_dir + '/dogs'

In [4]:
print('number of cats training images - ',len(os.listdir(train_dir_cats)))
print('number of dogs training images - ',len(os.listdir(train_dir_dogs)))
print('number of cats testing images - ',len(os.listdir(test_dir_cats)))
print('number of dogs testing images - ',len(os.listdir(test_dir_dogs)))

number of cats training images -  4000
number of dogs training images -  4000
number of cats testing images -  1000
number of dogs testing images -  1000


In [5]:
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)

In [6]:
batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (64, 64),
                                                  batch_size = batch_size,
                                                  class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 64)        0

In [9]:
fitted_model = model.fit(training_data,
                        steps_per_epoch = 250,
                        epochs = 50,
                        validation_data = testing_data,
                        validation_steps = 1000)

Epoch 1/50
250/250 [==============================] - 270s 1s/step - loss: 0.6940 - accuracy: 0.5089 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 2/50
250/250 [==============================] - 188s 751ms/step - loss: 0.6834 - accuracy: 0.5489
Epoch 3/50
250/250 [==============================] - 175s 699ms/step - loss: 0.6570 - accuracy: 0.6085
Epoch 4/50
250/250 [==============================] - 178s 714ms/step - loss: 0.6361 - accuracy: 0.6345
Epoch 5/50
250/250 [==============================] - 168s 670ms/step - loss: 0.6112 - accuracy: 0.6626
Epoch 6/50
250/250 [==============================] - 174s 697ms/step - loss: 0.5852 - accuracy: 0.6902
Epoch 7/50
250/250 [==============================] - 188s 751ms/step - loss: 0.5654 - accuracy: 0.7017
Epoch 8/50
250/250 [==============================] - 190s 761ms/step - loss: 0.5420 - accuracy: 0.7193
Epoch 9/50
250/250 [==============================] - 198s 793ms/step - loss: 0.5328 - accuracy: 0.7316
Epoch 10/50
250/250 [====

In [10]:
def predict_image(imagepath):
    predict = image.load_img(imagepath, target_size = (64, 64))   
    predict_modified = image.img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified, axis = 0)
    result = model.predict(predict_modified)
    if result[0][0] >= 0.5:
        prediction = 'cat'
        probability = result[0][0]
        print ("Prediction = " + prediction)
    else:
        prediction = 'dog'
        probability = 1 - result[0][0]
        print("Prediction = " + prediction)

In [11]:
print(predict_image(test_dir + '/dogs/dog.4079.jpg'))

Prediction = dog
None
